# Imports

In [ ]:
import requests
import pandas as pd

In [27]:
API_KEY = # your Binance API keys

SECRET_KEY = # you Binance API secret

# Data collection

## 1.1 Order books

In [54]:
# Define a function to get order book depth
def get_order_book(symbol, limit=10):
    base_url = "https://api.binance.com/api/v3/depth"
    params = {"symbol": symbol, "limit": limit}
    response = requests.get(base_url, params=params)
    data = response.json()
    return data

# Define a function to get historical price data (candlestick data)
def get_historical_data(symbol, interval='1h', limit=100):
    base_url = "https://api.binance.com/api/v3/klines"
    params = {"symbol": symbol, "interval": interval, "limit": limit}
    response = requests.get(base_url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'])
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
    df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
    return df



# Structure order book data 

def structure_order_book_data(symbol, data):
    return {
        "symbol": symbol,
        "lastUpdateId": data["lastUpdateId"],
        "bids": [{"price": float(bid[0]), "quantity": float(bid[1])} for bid in data["bids"]],
        "asks": [{"price": float(ask[0]), "quantity": float(ask[1])} for ask in data["asks"]]
    }


In [51]:
pairs = ["LINKUSDT", "AVAXUSDT", "TUSDUSDT", "XLMUSDT", "XMRUSDT", "ATOMUSDT", "UNIUSDT", "BUSDUSDT", "ETCUSDT", "HBARUSDT", "FILUSDT", "LDOUSDT", "ICPUSDT", "MKRUSDT", "APTUSDT", "VETUSDT", "OPUSDT", "ARBUSDT", "QNTUSDT", "NEARUSDT", "AAVEUSDT", "ALGOUSDT", "GRTUSDT", "STXUSDT","RNDRUSDT", "IMXUSDT"]
order_books = {}

for pair in pairs:
    raw_data = get_order_book(pair)
    structured_data = structure_order_book_data(pair, raw_data)
    order_books[pair] = structured_data


In [53]:
def analyze_order_book_in_usdt(order_book):
    best_bid = order_book['bids'][0]['price']
    best_ask = order_book['asks'][0]['price']
    
    spread = best_ask - best_bid
    spread_percentage = (spread / best_bid) * 100  # Calculate spread as percentage of best bid price
    
    bid_depth_usdt = sum([entry['price'] * entry['quantity'] for entry in order_book['bids']])
    ask_depth_usdt = sum([entry['price'] * entry['quantity'] for entry in order_book['asks']])
    
    imbalance = (bid_depth_usdt - ask_depth_usdt) / (bid_depth_usdt + ask_depth_usdt)
    
    return spread, spread_percentage, bid_depth_usdt, ask_depth_usdt, imbalance

analysis_results = []

for symbol, data in order_books.items():
    spread, spread_percentage, bid_depth_usdt, ask_depth_usdt, imbalance = analyze_order_book_in_usdt(data)
    analysis_results.append({
        "Symbol": symbol,
        "Spread": spread,
        "Spread Percentage": spread_percentage,
        "Bid Depth (USDT)": bid_depth_usdt,
        "Ask Depth (USDT)": ask_depth_usdt,
        "Imbalance": imbalance
    })

# Convert the results to a DataFrame
df = pd.DataFrame(analysis_results)

# Display the DataFrame
df


,Symbol,Spread,Spread Percentage,Bid Depth (USDT),Ask Depth (USDT),Imbalance
0,LINKUSDT,0.00100,0.013537,1.119524e+05,8.678462e+04,0.126639
1,AVAXUSDT,0.01000,0.103306,5.102172e+05,3.610560e+05,0.171199
2,TUSDUSDT,0.00010,0.010013,1.974786e+07,8.503274e+06,0.398023
3,XLMUSDT,0.00010,0.094251,4.415168e+05,3.211596e+05,0.157809
4,XMRUSDT,0.10000,0.064599,3.647243e+05,2.374065e+05,0.211445
5,ATOMUSDT,0.00100,0.014865,4.981856e+04,2.437743e+04,0.342891
6,UNIUSDT,0.00200,0.047642,7.797515e+04,5.158955e+04,0.203648
7,BUSDUSDT,0.00010,0.010003,6.448154e+06,1.402075e+07,-0.369956
8,ETCUSDT,0.01000,0.066711,4.302055e+05,3.215455e+05,0.144543
9,HBARUSDT,0.00010,0.214133,4.055116e+05,2.687112e+05,0.202901


## 1.2 Trade volume data

In [58]:
def get_recent_trades(symbol, limit=100):
    base_url = "https://api.binance.com/api/v3/trades"
    params = {"symbol": symbol, "limit": limit}
    response = requests.get(base_url, params=params)
    data = response.json()
    return data


In [59]:
recent_trades_data = {}

for pair in pairs:
    recent_trades_data[pair] = get_recent_trades(pair)


In [60]:
trade_volumes = {}

for pair, trades in recent_trades_data.items():
    total_volume = sum([float(trade['qty']) * float(trade['price']) for trade in trades])
    trade_volumes[pair] = total_volume

df_trade_volumes = pd.DataFrame(list(trade_volumes.items()), columns=['Pair', 'Total Volume'])


In [61]:
df_trade_volumes

,Pair,Total Volume
0,LINKUSDT,14347.470450
1,AVAXUSDT,33591.568400
2,TUSDUSDT,33616.625600
3,XLMUSDT,26608.575800
4,XMRUSDT,35035.392400
5,ATOMUSDT,21373.459740
6,UNIUSDT,11012.636210
7,BUSDUSDT,42654.788600
8,ETCUSDT,19438.581300
9,HBARUSDT,17229.901200


## 1.3 Historical data

In [62]:
from binance.client import Client
from datetime import datetime, timedelta
import time

# Initialize the Binance client
client = Client(api_key = API_KEY, api_secret = SECRET_KEY)

# List of pairs of interest

def get_historical_data(symbol):
    one_year_ago = datetime.now() - timedelta(days=365)
    timestamp_one_year_ago = int(time.mktime(one_year_ago.timetuple())) * 1000  # Convert to milliseconds
    historical_klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, str(timestamp_one_year_ago))
    
    df = pd.DataFrame(historical_klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Calculate the spread
    df['spread'] = df['high'].astype(float) - df['low'].astype(float)

    return df[['timestamp', 'open', 'high', 'low', 'close','volume', 'quote_asset_volume', 'spread']]


# Fetch historical data for each pair and store in a dictionary
historical_data = {}
for pair in pairs:
    historical_data[pair] = get_historical_data(pair)
    print(f"Fetched historical data for {pair} from {historical_data[pair]['timestamp'].iloc[0]} to {historical_data[pair]['timestamp'].iloc[-1]}")



Fetched historical data for LINKUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for AVAXUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for TUSDUSDT from 2023-03-11 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for XLMUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for XMRUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for ATOMUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for UNIUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for BUSDUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for ETCUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for HBARUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for FILUSDT from 2022-10-10 00:00:00 to 2023-10-09 00:00:00
Fetched historical data for LDOUSDT from 2022-10-10 00:00:00 to 2023-1

In [68]:
# Dictionary to store the results
results = {}

for pair, data in historical_data.items():
    # Convert the timestamp column to a datetime format for easy date operations
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    
    # Convert 'volume', 'close', and 'spread' columns to float
    data['quote_asset_volume'] = data['quote_asset_volume'].astype(float)
    data['close'] = data['close'].astype(float)
    
    results[pair] = {}
    for label, days in [('1 month', 30), ('6 months', 180), ('1 year', 365)]:
        
        # Filter data based on the timeframe
        end_date = data['timestamp'].max()
        start_date = end_date - timedelta(days=days)
        filtered_data = data[(data['timestamp'] >= start_date) & (data['timestamp'] <= end_date)]
        
        # Compute average daily trade volume
        avg_volume = filtered_data['quote_asset_volume'].mean()
        
        # Compute average spread
        
        # Calculate daily returns and then compute its standard deviation (volatility)
        filtered_data['daily_return'] = filtered_data['close'].pct_change()
        volatility = filtered_data['daily_return'].std()
        
        results[pair][label] = {
            'avg_daily_trade_volume': avg_volume,
            'volatility': volatility,
        }
        
# Storing the data in a pd df

data_list = []

for pair, data_dict in results.items():
    data_row = {'crypto': pair}
    
    for label, metrics in data_dict.items():
        if label == '1 month':
            suffix = '_1m'
        elif label == '6 months':
            suffix = '_6m'
        elif label == '1 year':
            suffix = '_1y'
        
        data_row[f'vol{suffix}'] = metrics['avg_daily_trade_volume']
        data_row[f'volatility{suffix}'] = metrics['volatility']
    
    data_list.append(data_row)
    
    
# Convert list to DataFrame
historical = pd.DataFrame(data_list)

# Reorder columns to match your preference
ordered_columns = ['crypto', 'vol_1m', 'vol_6m', 'vol_1y', 'volatility_1m', 'volatility_6m', 'volatility_1y']
historical = historical[ordered_columns]
    

<ipython-input-68-0c335c472229>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['daily_return'] = filtered_data['close'].pct_change()
<ipython-input-68-0c335c472229>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['daily_return'] = filtered_data['close'].pct_change()
<ipython-input-68-0c335c472229>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [69]:
historical

,crypto,vol_1m,vol_6m,vol_1y,volatility_1m,volatility_6m,volatility_1y
0,LINKUSDT,3.064590e+07,2.638773e+07,3.273021e+07,0.034570,0.036420,0.039028
1,AVAXUSDT,1.523686e+07,2.107744e+07,2.829402e+07,0.028467,0.030378,0.040913
2,TUSDUSDT,5.565308e+07,1.425269e+08,1.315071e+08,0.000236,0.001041,0.000992
3,XLMUSDT,6.037852e+06,9.824922e+06,7.905262e+06,0.019617,0.056319,0.045773
4,XMRUSDT,6.689424e+06,8.596945e+06,1.014374e+07,0.014084,0.017189,0.023350
5,ATOMUSDT,7.733207e+06,1.190960e+07,1.914354e+07,0.026931,0.025987,0.036429
6,UNIUSDT,3.896137e+06,7.145328e+06,1.029819e+07,0.020076,0.031749,0.037202
7,BUSDUSDT,8.068341e+07,1.502963e+08,4.479860e+08,0.000244,0.000231,0.000427
8,ETCUSDT,5.147595e+06,9.328795e+06,1.681857e+07,0.018598,0.028772,0.037485
9,HBARUSDT,5.143086e+06,7.368046e+06,7.563312e+06,0.022362,0.032498,0.040112


### 1.4 Merging relevant data

In [77]:
# Merge df with df_trade_volumes
df = df.merge(df_trade_volumes, left_on='Symbol', right_on='Pair', how='left').drop('Pair', axis=1)

# Merge df with df_volatility to get the 'volatility_1m' column
df = df.merge(historical, left_on='Symbol', right_on='crypto', how='left').drop('crypto', axis=1)

df = df.sort_values('Spread Percentage', ascending = False)

df = df[['Symbol','Spread','Spread Percentage','Bid Depth (USDT)','Ask Depth (USDT)','Imbalance']]

df

,Symbol,Spread,Spread Percentage,Bid Depth (USDT),Ask Depth (USDT),Imbalance
0,HBARUSDT,0.00010,0.214133,4.055116e+05,2.687112e+05,0.202901
1,GRTUSDT,0.00010,0.121951,2.422349e+05,9.546391e+04,0.434621
2,QNTUSDT,0.10000,0.114025,2.059589e+05,9.565337e+04,0.365720
3,ALGOUSDT,0.00010,0.103306,1.980116e+05,1.076319e+05,0.295703
4,AVAXUSDT,0.01000,0.103306,5.102172e+05,3.610560e+05,0.171199
5,NEARUSDT,0.00100,0.095785,2.446754e+05,1.085270e+05,0.385468
6,XLMUSDT,0.00010,0.094251,4.415168e+05,3.211596e+05,0.157809
7,OPUSDT,0.00100,0.081633,8.335162e+05,6.704001e+05,0.108461
8,MKRUSDT,1.00000,0.073746,3.395414e+05,3.053164e+05,0.053074
9,ETCUSDT,0.01000,0.066711,4.302055e+05,3.215455e+05,0.144543
